## Failed - Using pydantic to validate scraped data

In [38]:
%pip install requests
%pip install lxml
%pip install pandas
%pip install watermark
%pip install pydantic==2.3.0
%pip install pydantic-core==2.6.3
%pip install pydantic-extra-types
%pip install pycountry
%pip install word2number

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.5/374.5 KB 3.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 25.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.16.3
    Uninstalling pydantic_core-2.16.3:
      Successfully uninstalled pyda

#### Scrape the website content

In [58]:
from pydantic import (BaseModel, 
                      computed_field, 
                      field_serializer)
from pydantic_extra_types.currency_code import Currency
from urllib.parse import urljoin

class BookModel(BaseModel):
  prices: list[str]
  book_titles: list[str]
  book_urls: list[str]
  star_ratings_classes: list[str]

  @computed_field  
  @property
  def star_ratings(self) -> list[str]:
    return [star_rating.replace("star-rating ","") for star_rating in self.star_ratings_classes]

  @field_serializer('star_ratings')
  def serialize_star_rating(star_ratings: list[str]) -> list[int]:
    return [w2n.word_to_num(star_rating) for star_rating in star_ratings]

  @field_serializer('book_urls')
  def serialize_book_urls(book_urls: list[str]) -> list[HttpUrl]:
    return [urljoin("https://" , book_url) for book_url in book_urls]

In [61]:
import requests
from lxml import html
from word2number import w2n
import pandas
from typing import Union


class BookSpider:
    def __init__(self):
        self.session = requests.Session()
        self.headers = {
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'
            }
        self.bookmodel = BookModel

    def get(self):
        # loop the page number to scrape
        page_num = 1
        for page_num in range(0,50):
            url = f"https://books.toscrape.com/catalogue/page-{page_num}.html"
            print(url)
            output_data = self._fetch_data(url=url)
            validated_book_data = self._validate_book_data(output_data)
            print(validated_book_data) # need delete


    def _fetch_data(self, url):
        response = self.session.request("GET", url)
        tree = html.fromstring(response.content)      
        prices = tree.xpath("//article[@class='product_pod']//div[@class='product_price']\
            /p[@class='price_color']/text()")
        # bug: how to break down prices -> prices & currencies        
        book_titles = tree.xpath("//article[@class='product_pod']/h3/a/text()")
        book_urls = tree.xpath("//article[@class='product_pod']/h3/a/@href")
        star_ratings_classes = tree.xpath("//article[@class='product_pod']/p/@class")

        data_list = {
                    # "currencies": currencies,
                    "prices": prices,
                    "star_ratings_classes": star_ratings_classes,
                    "book_urls": book_urls, 
                    "book_titles": book_titles,
                    
                }
        return data_list

    def _validate_book_data(self, books_data: dict[str, Union[str, int, float]]) -> dict[str, list]:
        # validate book data with pydantic
        validated_book_data = dict(self.bookmodel.model_validate(books_data).model_dump()) # model_dump is to compute field
        return validated_book_data

book_spider = BookSpider()
book_spider.get()


https://books.toscrape.com/catalogue/page-0.html


{'prices': [], 'book_titles': [], 'book_urls': [], 'star_ratings_classes': [], 'star_ratings': []}
https://books.toscrape.com/catalogue/page-1.html
{'prices': ['£51.77', '£53.74', '£50.10', '£47.82', '£54.23', '£22.65', '£33.34', '£17.93', '£22.60', '£52.15', '£13.99', '£20.66', '£17.46', '£52.29', '£35.02', '£57.25', '£23.88', '£37.59', '£51.33', '£45.17'], 'book_titles': ['A Light in the ...', 'Tipping the Velvet', 'Soumission', 'Sharp Objects', 'Sapiens: A Brief History ...', 'The Requiem Red', 'The Dirty Little Secrets ...', 'The Coming Woman: A ...', 'The Boys in the ...', 'The Black Maria', 'Starving Hearts (Triangular Trade ...', "Shakespeare's Sonnets", 'Set Me Free', "Scott Pilgrim's Precious Little ...", 'Rip it Up and ...', 'Our Band Could Be ...', 'Olio', 'Mesaerion: The Best Science ...', 'Libertarianism for Beginners', "It's Only the Himalayas"], 'book_urls': ['https:///a-light-in-the-attic_1000/index.html', 'https:///tipping-the-velvet_999/index.html', 'https:///soumissi

/workspace/.pyenv_mirror/user/current/lib/python3.10/site-packages/pydantic/main.py:314: UserWarning: Pydantic serializer warnings:
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as expected
  Expected `url` but got `str` - serialized value may not be as e

{'prices': ['£12.84', '£37.32', '£30.52', '£25.27', '£34.53', '£54.64', '£22.50', '£53.13', '£40.30', '£44.18', '£17.66', '£31.05', '£23.82', '£36.89', '£15.94', '£33.29', '£18.02', '£19.63', '£52.22', '£33.63'], 'book_titles': ['In Her Wake', 'How Music Works', 'Foolproof Preserving: A Guide ...', 'Chase Me (Paris Nights ...', 'Black Dust', 'Birdsong: A Story in ...', "America's Cradle of Quarterbacks: ...", 'Aladdin and His Wonderful ...', 'Worlds Elsewhere: Journeys Around ...', 'Wall and Piece', 'The Four Agreements: A ...', 'The Five Love Languages: ...', 'The Elephant Tree', 'The Bear and the ...', "Sophie's World", 'Penny Maybe', 'Maude (1883-1993):She Grew Up ...', 'In a Dark, Dark ...', 'Behind Closed Doors', "You can't bury them ..."], 'book_urls': ['https:///in-her-wake_980/index.html', 'https:///how-music-works_979/index.html', 'https:///foolproof-preserving-a-guide-to-small-batch-jams-jellies-pickles-condiments-and-more-a-foolproof-guide-to-making-small-batch-jams-jellies-

## Computing environment

In [33]:
%load_ext watermark

%watermark

# print out pypi packages used
%watermark --iversions

# date
%watermark -u -n -t -z

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
Last updated: 2024-03-09T16:10:42.862195+00:00

Python implementation: CPython
Python version       : 3.10.12
IPython version      : 8.22.2

Compiler    : GCC 11.4.0
OS          : Linux
Release     : 6.1.75-060175-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

pandas     : 2.2.1
lxml       : 5.1.0
word2number: 1.1
requests   : 2.31.0

Last updated: Sat Mar 09 2024 16:10:42UTC

